<hr style="color:green" />
<h1 style="color:green">COSC2673 Assignment 2: Image Classification for Cancerous Cells</h1>
<h2 style="color:green">File 03: Basic SKLearn Decision Tree model test on Main data for Binary classification</h2>
<hr style="color:green" />

<p>
In this file, load the image dataset for the main labels, and try a basic binary classification according to the isCancerous flag. Afterwards, try the multi-class classification for the cell type
</p>

In [1]:
import pandas as pd
import numpy as np
import os
import cv2

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import data_basic_utility as dbutil
import graphing_utility as graphutil
import statistics_utility as statsutil
import a2_utility as a2util

randomSeed = dbutil.get_random_seed()

c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
dfImages = pd.read_csv("images_main.csv")

In [3]:
# Get The training Split and the Validation Split
dfImagesTrain = dfImages[dfImages["trainValTest"] == 0]
dfImagesVal = dfImages[dfImages["trainValTest"] == 1]

dfImagesTrain.head()

,ImageName,isCancerous,cellType,trainValTest
0,./Image_classification_data/patch_images\1.png,0,0,0
1,./Image_classification_data/patch_images\10.png,0,0,0
3,./Image_classification_data/patch_images\1000.png,1,2,0
4,./Image_classification_data/patch_images\10000...,0,1,0
5,./Image_classification_data/patch_images\10001...,0,1,0


In [4]:
# Load the images into a numpy array
imgsTrain = []
for imageName in dfImagesTrain["ImageName"]:
    img = cv2.imread(imageName)
    imgsTrain.append(img)
    

In [5]:
imgsTrain[0]

array([[[213, 187, 220],
        [214, 189, 220],
        [240, 215, 246],
        ...,
        [226, 197, 231],
        [207, 175, 210],
        [211, 183, 215]],

       [[209, 182, 216],
        [237, 210, 242],
        [254, 228, 255],
        ...,
        [217, 182, 223],
        [216, 178, 220],
        [231, 198, 237]],

       [[226, 197, 232],
        [252, 224, 255],
        [239, 211, 245],
        ...,
        [223, 190, 228],
        [222, 187, 227],
        [221, 188, 226]],

       ...,

       [[203, 151, 212],
        [193, 142, 204],
        [206, 154, 217],
        ...,
        [224, 187, 242],
        [230, 191, 247],
        [232, 190, 247]],

       [[217, 162, 222],
        [213, 161, 223],
        [205, 153, 216],
        ...,
        [221, 176, 233],
        [219, 174, 233],
        [224, 181, 240]],

       [[219, 161, 217],
        [210, 153, 214],
        [223, 167, 230],
        ...,
        [215, 169, 228],
        [213, 169, 228],
        [220, 177, 235]]

Each image is an array of 3-element arrays, corresponding to the RGB values. Flatten each image so that each image is a single array of numbers

In [6]:
imgsTrainFlat = [np.reshape(img, (-1,)) for img in imgsTrain]

In [7]:
for i in range(5):
    print(imgsTrainFlat[i])

[213 187 220 ... 220 177 235]
[238 198 240 ... 255 237 255]
[196 125 164 ... 255 238 253]
[225 200 216 ... 203 147 181]
[235 199 222 ... 198 142 184]


Do the same for the validation set. Now that we have the process, that code has been put into a utility file for reuse

In [8]:
imgsValFlat = a2util.loadImagesToFlattened(dfImagesVal, "ImageName")

Define our targets from the dataset

In [9]:
y_train_cancerous = dfImagesTrain["isCancerous"]
y_val_cancerous = dfImagesVal["isCancerous"]

Train a Decision Tree Classifier on the training data

In [10]:
modelCancerous = DecisionTreeClassifier()
modelCancerous.fit(imgsTrainFlat, y_train_cancerous)

DecisionTreeClassifier()

Predict on the validation set and generate results via confusion matrix

In [11]:
y_pred_cancerous = modelCancerous.predict(imgsValFlat)
cm = confusion_matrix(y_val_cancerous, y_pred_cancerous)

print("Binary Classification Results for isCancerous Predictions")
print(classification_report(y_val_cancerous, y_pred_cancerous))
print("F1 Score: " + str(f1_score(y_val_cancerous, y_pred_cancerous)))


Binary Classification Results for isCancerous Predictions
              precision    recall  f1-score   support

           0       0.62      0.79      0.69       408
           1       0.83      0.68      0.75       623

    accuracy                           0.72      1031
   macro avg       0.72      0.73      0.72      1031
weighted avg       0.75      0.72      0.73      1031

F1 Score: 0.7464539007092199


Using the basic Decision Tree, predictions on the images to find the binary result of whether the image was cancerous achieved a total F1 Score of **0.759**

Next, attempt another model to attempt to predict the multi classification cell type

In [13]:
# The targets for the multi classification model will be the cell type
y_train_celltype = dfImagesTrain["cellType"]
y_val_celltype = dfImagesVal["cellType"]

In [14]:
modelCellType = DecisionTreeClassifier()
modelCellType.fit(imgsTrainFlat, y_train_celltype)

DecisionTreeClassifier()

For the Cell type model, Predict on the validation set and generate results via confusion matrix.

When generating the F1 Score, for simplicity, a singular F1 score will be reported on. This single score will use the "micro" approach of averaging, which calculate metrics globally by counting the total true positives, false negatives and false positives.

In [18]:
y_pred_celltype = modelCellType.predict(imgsValFlat)
cm = confusion_matrix(y_val_celltype, y_pred_celltype)

print("Binary Classification Results for Cell Type Predictions")
print(classification_report(y_val_celltype, y_pred_celltype))
print("F1 Score (Average=micro): " + str(f1_score(y_val_celltype, y_pred_celltype, average="micro")))


Binary Classification Results for isCancerous Predictions
              precision    recall  f1-score   support

           0       0.35      0.37      0.36       155
           1       0.44      0.54      0.48       185
           2       0.82      0.67      0.74       623
           3       0.07      0.13      0.09        68

    accuracy                           0.57      1031
   macro avg       0.42      0.43      0.42      1031
weighted avg       0.63      0.57      0.59      1031

F1 Score per class: [0.35913313 0.4842615  0.74093722 0.09230769]
F1 Score: 0.5683802133850631


As expected, a basic decision tree for image classification performs poorly in general. The Multi-classification model performs particularly poorly, where it identifiers class 2 with a precision of 0.82, but has very low precision for the other classes, and a low overall F1 Score of 0.568